In [1]:
%config Completer.use_jedi = False
import pandas as pd
import numpy as np
import openpyxl
#import lifelines

In [95]:
data=pd.read_excel('data_after_cleaned.xlsx',engine='openpyxl')
action_to_drop=['בירור- אמבולנס','איחוד הצלה - עודכנו','אמבולנס - ביטול']
data=data.loc[~data['action_desc'].isin(action_to_drop)]
x=data.drop(['action_desc',
             'counts',
             'quantile_10_count',
             'out_std',
             'q_out',
             't_call_int',
             'count_until_t_Call'],axis=1)
y=x.drop_duplicates()
print(len(y))
df=y.copy()

12664


In [3]:
chr(97).upper()

'A'

In [79]:
map_dict={}
i=97
for act in list(df['act_category'].unique()):
    map_dict[act]=chr(i).upper()
    i=i+1

In [81]:
df['act_category_code']=df['act_category'].map(map_dict)

In [82]:
df['seq']=0
df['name_seq']=0
for index, row in df.iterrows():
    df.loc[df['id']==row['id'],'seq']=df.loc[df['id']==row['id']]['act_category_code'].str.cat(sep=',')
    df.loc[df['id']==row['id'],'name_seq']=df.loc[df['id']==row['id']]['act_category'].str.cat(sep=',')
    #for index1, row in df.iterrows():


In [96]:
df['seq']=0
df['name_seq']=0
l=[]
id_list=[]
k=1
for index, row in df.iterrows():
    if row['id'] in id_list:
        continue
    id_list.append(row['id'])
    for i in list(df[df['id']==row['id']]['year'].unique()):
        l.append(df.loc[(df['id']==row['id'])&(df['year']==i)]['act_category'].str.cat(sep=','))
        
    #df.loc[df['id']==row['id'],'seq']=df.loc[df['id']==row['id']]['act_category_code'].str.cat(sep=',')
    #df.loc[df['id']==row['id'],'name_seq']=df.loc[df['id']==row['id']]['act_category'].str.cat(sep=',')
    #for index1, row in df.iterrows():


In [36]:
(len(l),)
#df[['id','year']].value_counts()

(2335,)

In [97]:
new=[]
for i in l:
    x=tuple(map(str, i.split(',')))
    new.append(x)
    

In [98]:
new1=tuple(new)

In [46]:
# new1

In [38]:
res = tuple(map(str, l[0].split(',')))
res

('שירותי רופא',
 'ייעוץ טלפוני',
 'בירור',
 'שירותי רופא',
 'ייעוץ טלפוני',
 'שירותי רופא',
 'ייעוץ טלפוני')

In [76]:
l
df[(df['id']==241436)&(df['year']==2012)]

,residence_type,sug_erua,t_call,month,hour,year,day_type,day,id,gender_code,gender_name,lon_code,lon_desc,act_category,type_category,season,act_category_code,seq,name_seq
0,קהילה,ECI לחצן,2012-01-01 08:14:03.653,1,8,2012,יום חול,ראשון,241436,6,זוג,2,בן זוג,שירותי רופא,שירותי בריאות,חורף,A,0,0
968,קהילה,ECI לחצן,2012-04-18 14:22:13.700,4,14,2012,יום חול,רביעי,241436,6,זוג,2,בן זוג,ייעוץ טלפוני,קריאות חירום - שירותים כלליים,אביב,B,0,0
971,קהילה,ECI לחצן,2012-04-18 14:53:56.620,4,14,2012,יום חול,רביעי,241436,6,זוג,2,בן זוג,בירור,קריאות חירום - שירותים כלליים,אביב,K,0,0
972,קהילה,ECI מכשיר,2012-04-18 14:55:23.583,4,14,2012,יום חול,רביעי,241436,6,זוג,2,בן זוג,שירותי רופא,שירותי בריאות,אביב,A,0,0
1929,קהילה,ECI לחצן,2012-08-07 20:14:08.063,8,20,2012,יום חול,שלישי,241436,6,זוג,2,בן זוג,ייעוץ טלפוני,קריאות חירום - שירותים כלליים,קיץ,B,0,0
3336,קהילה,ECI לחצן,2012-12-13 14:59:26.560,12,14,2012,יום חול,חמישי,241436,6,זוג,2,בן זוג,שירותי רופא,שירותי בריאות,חורף,A,0,0
3337,קהילה,ECI לחצן,2012-12-13 14:59:26.560,12,14,2012,יום חול,חמישי,241436,6,זוג,2,בן זוג,ייעוץ טלפוני,קריאות חירום - שירותים כלליים,חורף,B,0,0


In [80]:
# df[df['id']==7859]

In [83]:
seq_df=df[['id','seq','name_seq']].drop_duplicates()
len(seq_df)

775

In [84]:
seq_df['tupple']=seq_df['seq'].apply(lambda x:tuple(x.split(',')))
seq_df['tupple_name']=seq_df['name_seq'].apply(lambda x:tuple(x.split(',')))
tap=[]
for index, row in seq_df.iterrows():
    tap.append(row['tupple_name'])
tap1=tuple(tap)


In [19]:
# tap1

In [20]:
from pymining import seqmining
from pymining import itemmining, assocrules, perftesting

In [99]:
transactions = new1
relim_input = itemmining.get_relim_input(transactions)
item_sets = itemmining.relim(relim_input, min_support=100)
rules = assocrules.mine_assoc_rules(item_sets, min_support=100, min_confidence=0.5)

In [100]:
df_seq = pd.DataFrame(list(rules), columns=['gorerr', 'novea','support','confidence'])

In [87]:
transactions = tap1
relim_input = itemmining.get_relim_input(transactions)
item_sets = itemmining.relim(relim_input, min_support=100)
rules = assocrules.mine_assoc_rules(item_sets, min_support=100, min_confidence=0.7)

In [102]:
df_seq.head(88)

,gorerr,novea,support,confidence
0,"(פנייה לסיוע כללי, בירור, שירותי רופא)",(ייעוץ טלפוני),110,0.940171
1,"(פנייה לסיוע כללי, בירור)","(ייעוץ טלפוני, שירותי רופא)",110,0.666667
2,"(ייעוץ טלפוני, פנייה לסיוע כללי, בירור)",(שירותי רופא),110,0.743243
3,"(פעולות איש קשר, בירור, שירותי רופא)",(ייעוץ טלפוני),126,0.954545
4,"(בירור, שירותי רופא)","(ייעוץ טלפוני, פעולות איש קשר)",126,0.506024
...,...,...,...,...
83,(פנייה לסיוע כללי),(ייעוץ טלפוני),421,0.627422
84,(פעולות איש קשר),(שירותי רופא),572,0.826590
85,(פעולות איש קשר),(ייעוץ טלפוני),509,0.735549
86,(שירותי רופא),(ייעוץ טלפוני),770,0.667823


In [94]:
# rules

In [89]:
df_seq = pd.DataFrame(list(rules), columns=['gorerr', 'novea','support','confidence'])

In [103]:
df_seq.to_csv('saoundperyear.csv',index=False,encoding = 'utf-8-sig')

In [93]:
df_seq.to_csv('saound.csv',index=False,encoding = 'utf-8-sig')

In [69]:
# from pymining import seqmining
# seqs = tap1[1:10]
# freq_seqs = seqmining.freq_seq_enum(seqs, 2)
# sorted(freq_seqs)

In [ ]:
from pymining import itemmining, assocrules, perftesting
transactions = perftesting.get_default_transactions()
relim_input = itemmining.get_relim_input(transactions)
item_sets = itemmining.relim(relim_input, min_support=2)
rules = assocrules.mine_assoc_rules(item_sets, min_support=2, min_confidence=0.5)
rules